# Parquet Files with Python

This notebook demonstrates how to:
- Create Parquet database files
- Query data from Parquet files
- Update and append data to Parquet files
- Use pandas, pyarrow, and DuckDB for efficient Parquet operations

## Installation

Install required libraries:
```bash
pip install pandas pyarrow fastparquet duckdb
```

**Libraries:**
- `pandas` - Data manipulation and analysis
- `pyarrow` - Apache Arrow implementation for reading/writing Parquet
- `fastparquet` - Alternative Parquet implementation
- `duckdb` - SQL queries on Parquet files without loading into memory

In [2]:
# Import required libraries
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import duckdb
from pathlib import Path

print("✅ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"PyArrow version: {pa.__version__}")
print(f"DuckDB version: {duckdb.__version__}")

✅ Libraries imported successfully
Pandas version: 2.3.3
PyArrow version: 22.0.0
DuckDB version: 1.4.3


## 1. Create Sample Data

First, let's create sample data to work with. We'll create a dataset of employees with various attributes.

In [3]:
# Create sample employee data
data = {
    'employee_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'name': ['Alice Johnson', 'Bob Smith', 'Charlie Brown', 'Diana Prince', 
             'Eve Davis', 'Frank Miller', 'Grace Lee', 'Henry Wilson', 
             'Iris Chen', 'Jack Taylor'],
    'department': ['Engineering', 'Sales', 'Engineering', 'HR', 'Sales', 
                   'Engineering', 'HR', 'Sales', 'Engineering', 'HR'],
    'salary': [95000, 65000, 88000, 72000, 70000, 105000, 68000, 75000, 92000, 71000],
    'years_experience': [5, 3, 4, 6, 2, 8, 4, 5, 6, 3],
    'remote': [True, False, True, False, True, True, False, False, True, False]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the data
print("Sample Employee Data:")
print(df)
print(f"\nShape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Sample Employee Data:
   employee_id           name   department  salary  years_experience  remote
0            1  Alice Johnson  Engineering   95000                 5    True
1            2      Bob Smith        Sales   65000                 3   False
2            3  Charlie Brown  Engineering   88000                 4    True
3            4   Diana Prince           HR   72000                 6   False
4            5      Eve Davis        Sales   70000                 2    True
5            6   Frank Miller  Engineering  105000                 8    True
6            7      Grace Lee           HR   68000                 4   False
7            8   Henry Wilson        Sales   75000                 5   False
8            9      Iris Chen  Engineering   92000                 6    True
9           10    Jack Taylor           HR   71000                 3   False

Shape: (10, 6)
Columns: ['employee_id', 'name', 'department', 'salary', 'years_experience', 'remote']


## 2. Write Data to Parquet File

Parquet is a columnar storage format that provides efficient compression and encoding. It's ideal for analytical queries and big data processing.

In [4]:
# Define file path
parquet_file = 'employees.parquet'

# Method 1: Using pandas (simple and common)
df.to_parquet(parquet_file, engine='pyarrow', compression='snappy', index=False)
print(f"✅ Data written to {parquet_file}")

# Check file size
file_size = Path(parquet_file).stat().st_size
print(f"File size: {file_size:,} bytes ({file_size/1024:.2f} KB)")

# Method 2: Using PyArrow directly (more control)
table = pa.Table.from_pandas(df)
pq.write_table(table, 'employees_pyarrow.parquet', compression='snappy')
print("✅ Also created employees_pyarrow.parquet using PyArrow directly")

✅ Data written to employees.parquet
File size: 4,219 bytes (4.12 KB)
✅ Also created employees_pyarrow.parquet using PyArrow directly


## 3. Read Data from Parquet File

Reading Parquet files is straightforward and efficient, especially for large datasets.

In [ ]:
# Method 1: Read entire file with pandas
df_read = pd.read_parquet(parquet_file, engine='pyarrow')
print("Data read from Parquet file:")
print(df_read)

# Method 2: Read only specific columns (efficient!)
df_subset = pd.read_parquet(parquet_file, columns=['name', 'department', 'salary'])
print("\n📌 Reading only specific columns:")
print(df_subset.head())

# Method 3: Using PyArrow for more control
table = pq.read_table(parquet_file)
print(f"\n📊 Schema information:")
print(table.schema)

## 4. Query Data with Pandas

Use pandas filtering and query methods to select specific data from Parquet files.

In [ ]:
# Read the data
df = pd.read_parquet(parquet_file)

# Query 1: Filter by department
engineering = df[df['department'] == 'Engineering']
print("🔍 Query 1: Engineering employees")
print(engineering[['name', 'department', 'salary']])

# Query 2: Filter by salary range
high_earners = df[df['salary'] >= 90000]
print("\n🔍 Query 2: Employees earning >= $90,000")
print(high_earners[['name', 'salary', 'department']])

# Query 3: Multiple conditions
experienced_remote = df[(df['years_experience'] >= 5) & (df['remote'] == True)]
print("\n🔍 Query 3: Remote employees with 5+ years experience")
print(experienced_remote[['name', 'years_experience', 'remote']])

# Query 4: Using .query() method (SQL-like syntax)
sales_dept = df.query("department == 'Sales' and salary > 65000")
print("\n🔍 Query 4: Sales employees earning > $65,000")
print(sales_dept[['name', 'salary']])

# Query 5: Aggregations
print("\n📊 Query 5: Average salary by department")
print(df.groupby('department')['salary'].agg(['mean', 'count', 'min', 'max']))

## 5. Query Data with SQL (DuckDB)

DuckDB allows you to run SQL queries directly on Parquet files without loading them entirely into memory. This is extremely efficient for large datasets.

In [ ]:
# Create DuckDB connection
conn = duckdb.connect(':memory:')  # In-memory database

# SQL Query 1: SELECT all data
result = conn.execute(f"SELECT * FROM '{parquet_file}'").df()
print("🔍 SQL Query 1: SELECT all rows")
print(result)

# SQL Query 2: Filter by department
query2 = f"""
    SELECT name, department, salary 
    FROM '{parquet_file}' 
    WHERE department = 'Engineering'
    ORDER BY salary DESC
"""
result2 = conn.execute(query2).df()
print("\n🔍 SQL Query 2: Engineering department (sorted by salary)")
print(result2)

# SQL Query 3: Aggregations with GROUP BY
query3 = f"""
    SELECT 
        department,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary,
        MAX(salary) as max_salary
    FROM '{parquet_file}'
    GROUP BY department
    ORDER BY avg_salary DESC
"""
result3 = conn.execute(query3).df()
print("\n🔍 SQL Query 3: Department statistics")
print(result3)

# SQL Query 4: Complex query with multiple conditions
query4 = f"""
    SELECT name, salary, years_experience, remote
    FROM '{parquet_file}'
    WHERE salary > 70000 
      AND (remote = true OR years_experience >= 5)
    ORDER BY salary DESC
"""
result4 = conn.execute(query4).df()
print("\n🔍 SQL Query 4: High earners who are remote or experienced")
print(result4)

# SQL Query 5: JOIN example (self-join to compare salaries)
query5 = f"""
    SELECT 
        department,
        COUNT(CASE WHEN salary > 80000 THEN 1 END) as high_earners,
        COUNT(*) as total_employees
    FROM '{parquet_file}'
    GROUP BY department
"""
result5 = conn.execute(query5).df()
print("\n🔍 SQL Query 5: High earners (>$80k) by department")
print(result5)

## 6. Update Data in Parquet File

Parquet files are immutable (read-only), so "updating" means reading the file, modifying the data in memory, and writing it back to the same file (or a new file).

In [ ]:
# Read existing data
df = pd.read_parquet(parquet_file)
print("Original data:")
print(df[['employee_id', 'name', 'salary']])

# Update 1: Give everyone in Engineering a 10% raise
df.loc[df['department'] == 'Engineering', 'salary'] = \
    df.loc[df['department'] == 'Engineering', 'salary'] * 1.10

# Update 2: Update specific employee by ID
df.loc[df['employee_id'] == 5, 'years_experience'] = 3  # Eve got promoted

# Update 3: Add a new column
df['bonus'] = df['salary'] * 0.10  # 10% bonus for all

print("\n✅ Updated data:")
print(df[['employee_id', 'name', 'salary', 'bonus']])

# Write updated data back to file
df.to_parquet(parquet_file, engine='pyarrow', compression='snappy', index=False)
print(f"\n✅ Updated data written back to {parquet_file}")

# Verify the update
df_verify = pd.read_parquet(parquet_file)
print("\nEngineering salaries after 10% raise:")
print(df_verify[df_verify['department'] == 'Engineering'][['name', 'salary']])

## 7. Append New Data to Parquet File

Add new rows to an existing Parquet file by reading it, concatenating new data, and writing back.

In [ ]:
# Create new employee records
new_employees = pd.DataFrame({
    'employee_id': [11, 12, 13],
    'name': ['Karen White', 'Leo Martinez', 'Maya Patel'],
    'department': ['Engineering', 'Sales', 'HR'],
    'salary': [98000, 72000, 74000],
    'years_experience': [7, 4, 5],
    'remote': [True, False, True],
    'bonus': [9800, 7200, 7400]  # Include the bonus column we added
})

print("New employees to add:")
print(new_employees)

# Read existing data
df_existing = pd.read_parquet(parquet_file)
print(f"\nCurrent number of employees: {len(df_existing)}")

# Append new data
df_combined = pd.concat([df_existing, new_employees], ignore_index=True)
print(f"After append: {len(df_combined)} employees")

# Write combined data back
df_combined.to_parquet(parquet_file, engine='pyarrow', compression='snappy', index=False)
print(f"✅ New employees added to {parquet_file}")

# Verify
df_verify = pd.read_parquet(parquet_file)
print("\nAll employees (showing last 5):")
print(df_verify.tail()[['employee_id', 'name', 'department', 'salary']])

## 8. Delete Data from Parquet File

Remove specific rows by filtering them out and writing the remaining data back to the file.

In [ ]:
# Read current data
df = pd.read_parquet(parquet_file)
print(f"Current employee count: {len(df)}")
print("\nCurrent employees:")
print(df[['employee_id', 'name', 'department']])

# Delete 1: Remove specific employee by ID
df_filtered = df[df['employee_id'] != 12]  # Remove Leo Martinez (ID 12)

# Delete 2: Remove all employees from a specific department
# df_filtered = df_filtered[df_filtered['department'] != 'HR']  # Uncomment to remove HR

print(f"\nAfter deletion: {len(df_filtered)} employees")
print(df_filtered[['employee_id', 'name', 'department']])

# Write back to file
df_filtered.to_parquet(parquet_file, engine='pyarrow', compression='snappy', index=False)
print(f"\n✅ Employee(s) removed from {parquet_file}")

# Verify deletion
df_verify = pd.read_parquet(parquet_file)
print(f"\nVerified: {len(df_verify)} employees remain")
print("Employee IDs:", df_verify['employee_id'].tolist())

## 9. Partitioned Parquet Files (Advanced)

For large datasets, you can partition Parquet files by column values (e.g., by department). This creates separate files for each partition, enabling faster queries when filtering by the partition column.

In [ ]:
# Read data
df = pd.read_parquet(parquet_file)

# Write partitioned by department
partition_dir = 'employees_partitioned'
df.to_parquet(
    partition_dir,
    engine='pyarrow',
    partition_cols=['department'],  # Creates separate files per department
    compression='snappy',
    index=False
)
print(f"✅ Created partitioned Parquet files in '{partition_dir}/' directory")

# List partition directories
import os
if os.path.exists(partition_dir):
    for item in os.listdir(partition_dir):
        print(f"  📁 {item}")

# Read from partitioned dataset - only reads relevant partitions
df_engineering = pd.read_parquet(
    partition_dir,
    filters=[('department', '==', 'Engineering')]  # Only reads Engineering partition
)
print(f"\n🔍 Read only Engineering partition: {len(df_engineering)} rows")
print(df_engineering[['name', 'salary']])

# Read entire partitioned dataset
df_all = pd.read_parquet(partition_dir)
print(f"\n📊 Total rows from all partitions: {len(df_all)}")

## Summary

This notebook demonstrated how to:

✅ **Create Parquet files** - Using pandas with PyArrow backend
✅ **Read Parquet files** - Full reads, column selection, and schema inspection
✅ **Query with Pandas** - Filtering, aggregations, and the `.query()` method
✅ **Query with SQL** - Using DuckDB for SQL queries directly on Parquet files
✅ **Update data** - Modify values and add columns
✅ **Append data** - Add new rows to existing files
✅ **Delete data** - Remove rows by filtering
✅ **Partition data** - Create partitioned datasets for efficient querying

### Key Advantages of Parquet:
- **Columnar storage** - Only read columns you need
- **Compression** - Smaller file sizes than CSV
- **Type preservation** - Data types are stored, no parsing needed
- **Fast queries** - Especially with DuckDB or partitioning
- **Schema evolution** - Can add/remove columns over time

### Best Practices:
1. Use `compression='snappy'` for good balance of speed and size
2. Partition large datasets by frequently-filtered columns
3. Use DuckDB for SQL queries on large Parquet files (avoids loading into memory)
4. Read only necessary columns with `columns=['col1', 'col2']`
5. For production, consider using a proper database or data lake

### Next Steps:
- Integrate with Apache Spark for very large datasets
- Use with cloud storage (S3, Azure Blob, GCS)
- Implement incremental updates with Delta Lake or Apache Iceberg
- Create data pipelines with Parquet as the storage format